<a href="https://colab.research.google.com/github/it-is-nidah/dataset/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 3

Q1 Develop a CNN from scratch for detection of GAN generated images. Training dataset (150 live
and 150 fake images) should be used for CNN training. Trained CNN should be evaluated on test
dataset (150 live and 150 fake images). Report Accuracy, F1-score and Recall of the classifier.
Dataset:
https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces

In [ ]:
import pandas as pd
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!unzip gdrive/My\ Drive/dataset.zip

Archive:  gdrive/My Drive/dataset.zip
replace real_vs_fake/real-vs-fake/test/fake/00276TOPP4.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
def preprocess_data():
  real_images_path = "/content/real_vs_fake/real-vs-fake/train/real"
  fake_images_path = "/content/real_vs_fake/real-vs-fake/train/fake"
  
  real_images = []
  fake_images = []

  for i in range(1, 151):
      real_img = cv2.imread(real_images_path + str(i) + ".jpg")
      fake_img = cv2.imread(fake_images_path + str(i) + ".jpg")

      real_img = cv2.resize(real_img, (128, 128))
      fake_img = cv2.resize(fake_img, (128, 128))

      real_images.append(real_img)
      fake_images.append(fake_img)

  real_labels = np.ones(len(real_images))
  fake_labels = np.zeros(len(fake_images))

  X = np.concatenate((real_images, fake_images), axis=0)
  y = np.concatenate((real_labels, fake_labels), axis=0)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

  return X_train, X_test, y_train, y_test

In [ ]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

    return model

In [ ]:
def train_model(model, X_train, y_train):
    history = model.fit(X_train)

Question 2: Use the same CNN architecture trained on any image dataset of your choice. Fine-tune the CNN
model on the 150 live and 150 fake images (from the above data source) for the detection of GAN
generated images. Fine-tuned CNN should be tested on 150 live and 150 fake images. Report the
difference in accuracy, F1-score and recall of the classifier from those observed in Question #1.

In [ ]:
def preprocess_data():
    real_images_path = "/content/real_vs_fake/real-vs-fake/test/real/"
    fake_images_path = "/content/real_vs_fake/real-vs-fake/test/fake"

    real_images = []
    fake_images = []


    for i in range(1, 151):
        real_img = cv2.imread(real_images_path + str(i) + ".jpg")
        fake_img = cv2.imread(fake_images_path + str(i) + ".jpg")

        real_img = cv2.resize(real_img, (224, 224))
        fake_img = cv2.resize(fake_img, (224, 224))

        real_images.append(real_img)
        fake_images.append(fake_img)

    real_labels = np.ones(len(real_images))
    fake_labels = np.zeros(len(fake_images))

    X = np.concatenate((real_images, fake_images), axis=0)
    y = np.concatenate((real_labels, fake_labels), axis=0)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

    return X_train, X_test, y_train, y_test


In [ ]:
def load_model():
  vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

  x = vgg_model.output
  x = Flatten()(x)
  x = Dense(256, activation='relu')(x)
  x = Dense(128, activation='relu')(x)
  predictions = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=vgg_model.input, outputs=predictions)

  for layer in vgg_model.layers:
      layer.trainable = False

  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])